In [4]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from Dataset.RNN_Economy_Dataset import RNN_Economy_Dataset
from Dataset.RNN_Transaction_Dataset import RNN_Transaction_Dataset

from Model.LSTM import LSTM

In [5]:
input_size = 5
hidden_size = 256
output_size = 1

lr = 1e-3
num_epochs = 100

In [6]:
economy_df = pd.read_excel('../데이터/Economy/economy_all.xlsx')
transaction_df = pd.read_csv('../데이터/Transaction/transaction_final.csv', index_col=0)

In [11]:
# economy_train_size = int(len(economy_df)*0.7)
# economy_val_size = int(len(economy_df)*0.3)
trainsaction_train_size = int(len(transaction_df)*0.7)
trainsaction_val_size = int(len(transaction_df)*0.3)

# economy_train_dataset = RNN_Economy_Dataset(economy_df[:economy_train_size])
# economy_train_loader = DataLoader(economy_train_dataset, batch_size=2)
# economy_val_dataset = RNN_Economy_Dataset(economy_df[economy_train_size:economy_train_size+economy_val_size])
# economy_val_loader = DataLoader(economy_val_dataset, batch_size=2)

transaction_train_dataset = RNN_Transaction_Dataset(transaction_df[:trainsaction_train_size])
transaction_train_loader = DataLoader(transaction_train_dataset, batch_size=2, drop_last=True)
transaction_val_dataset = RNN_Transaction_Dataset(transaction_df[trainsaction_train_size:])
transaction_val_loader = DataLoader(transaction_val_dataset, batch_size=2, drop_last=True)

/Users/kimhakhyun/Desktop/Git/sci/SCI/코드/Dataset/RNN_Transaction_Dataset.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['계약년월'] = pd.to_datetime(data['계약년월'])
/Users/kimhakhyun/Desktop/Git/sci/SCI/코드/Dataset/RNN_Transaction_Dataset.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['계약년월'] = pd.to_datetime(data['계약년월'])


In [12]:
model = LSTM(input_size, hidden_size, output_size)

In [13]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

### 경제 모델

In [19]:
best_val_loss = float('inf') 
for epoch in range(num_epochs + 1):
    model.train()
    for batch_idx, samples in enumerate(economy_train_loader):
        nation_x_train, nation_y_train, call_x_train, call_y_train = samples

        prediction = model(nation_x_train)
        cost = criterion(prediction, nation_y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_idx, samples in enumerate(economy_val_loader):
            nation_x_val, nation_y_val, call_x_val, call_y_val = samples

            prediction = model(nation_x_val)
            loss = criterion(prediction, nation_y_val)
            val_loss += loss.item()

    val_loss /= len(economy_val_loader)
    print(f'Epoch {epoch}/{num_epochs}, Training Loss: {cost.item()}, Validation Loss: {val_loss}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'Checkpoint/best_rnn_economy_model.pth')

Epoch 0/100, Training Loss: 6.587806274183095e-05, Validation Loss: 0.823525653087667
Epoch 1/100, Training Loss: 0.00042392080649733543, Validation Loss: 0.834231839606738
Epoch 2/100, Training Loss: 0.0008335366146638989, Validation Loss: 0.8368861662706227
Epoch 3/100, Training Loss: 0.0012891507940366864, Validation Loss: 0.8408586972925279
Epoch 4/100, Training Loss: 0.0016851091058924794, Validation Loss: 0.8404338035366631
Epoch 5/100, Training Loss: 0.0020511806942522526, Validation Loss: 0.8398158955387771
Epoch 6/100, Training Loss: 0.0023386471439152956, Validation Loss: 0.8367820124008826
Epoch 7/100, Training Loss: 0.002581319771707058, Validation Loss: 0.8331335077860526
Epoch 8/100, Training Loss: 0.0027590461540967226, Validation Loss: 0.8277779057555433
Epoch 9/100, Training Loss: 0.00289554288610816, Validation Loss: 0.8216467336751521
Epoch 10/100, Training Loss: 0.002982519334182143, Validation Loss: 0.8139071623528642
Epoch 11/100, Training Loss: 0.0030447675380855

### 부동산 모델

In [14]:
best_val_loss = float('inf') 
for epoch in range(num_epochs + 1):
    model.train()
    for batch_idx, samples in enumerate(transaction_train_loader):
        dong_x_train, dong_y_train = samples

        prediction = model(dong_x_train)
        cost = criterion(prediction, dong_y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_idx, samples in enumerate(transaction_val_loader):
            x_val, y_val = samples

            prediction = model(x_val)
            loss = criterion(prediction, y_val)
            val_loss += loss.item()

    val_loss /= len(transaction_val_loader)
    print(f'Epoch {epoch}/{num_epochs}, Training Loss: {cost.item()}, Validation Loss: {val_loss}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'Checkpoint/best_rnn_transaction_model.pth')

RuntimeError: stack expects each tensor to be equal size, but got [4094, 5] at entry 0 and [7956, 5] at entry 1